# Data analysis

In [1]:
import requests
import json
from pprint import pprint
import pandas as pd 
import re

In [2]:
url = "https://firestore.googleapis.com/v1/projects/mimirhome-app/databases/(default)/documents/"

In [45]:
# Example - get all readings for a certain user/ device
import requests
import json
from pprint import pprint

# Receive a Radar Mode Request from a User
# They will have to select a device and a plant that they want a reading for
deviceId = "P4006" # should be unique
userId = "xyz" # would ordinarily get a user ID as well

# They would say which species they are performing the analysis for
plant_species = "aloe vera" # would normally be a plant ID

# Build up a URL for request
base_url = "https://firestore.googleapis.com/v1/projects/mimirhome-app/databases/(default)/documents/"

# Get the relevant sensor data
sensor_data = requests.get(base_url + "mimirReading/").text

# Obtain in json format
sensor_json = json.loads(sensor_data)


In [46]:
len(sensor_json['documents'])

20

In [30]:
sensor_json["nextPageToken"]

'AFTOeJwpaPh60H5tbmVoMX1aotff783KaArDc2o7i_b_lYtJEvzyIFpACKysXWOsfG5kl9Q_MsukMa64L7wCUXWf6jrn4_57DRsBmSBoy2n-WzOHn87JEVH4quImLidjhTwzNgA7eA'

In [56]:
sensor_json['documents'][0]['fields']

{'serialNumber': {'stringValue': 'VFC7uk23p7z'},
 'iaq': {'doubleValue': 49.31381},
 'timestamp': {'integerValue': '1600401016692'},
 'eVOC': {'doubleValue': 0.922448},
 'bearing': {'integerValue': '-136'},
 'pressure': {'integerValue': '97263'},
 'humidity': {'doubleValue': 51.22694},
 'altitude': {'doubleValue': 343.8147},
 'batteryPercent': {'integerValue': '97'},
 'eCO2': {'doubleValue': 662.0548},
 'deviceId': {'stringValue': 'So15xXTTj0GAoVeomUCA'},
 'luminance': {'integerValue': '0'},
 'iaqAccuracy': {'integerValue': '3'},
 'temperature': {'doubleValue': 24.47485},
 'userId': {'stringValue': 'ztx11HwFoJPuDJd1p7lLilvvkjg2'}}

In [58]:
df = pd.DataFrame(sensor_json['documents'][0]['fields'])

In [79]:
df = df.ffill()

In [80]:
df = df.backfill()

In [88]:
df = df.drop(df.index[[1,2]])

,serialNumber,iaq,timestamp,eVOC,bearing,pressure,humidity,altitude,batteryPercent,eCO2,deviceId,luminance,iaqAccuracy,temperature,userId
stringValue,VFC7uk23p7z,49.31381,1600401016692,0.922448,-136,97263,51.22694,343.8147,97,662.0548,So15xXTTj0GAoVeomUCA,0,3,24.47485,ztx11HwFoJPuDJd1p7lLilvvkjg2


In [89]:
df.columns

Index(['serialNumber', 'iaq', 'timestamp', 'eVOC', 'bearing', 'pressure',
       'humidity', 'altitude', 'batteryPercent', 'eCO2', 'deviceId',
       'luminance', 'iaqAccuracy', 'temperature', 'userId'],
      dtype='object')

In [120]:
column_names = ['serialNumber', 'iaq', 'timestamp', 'eVOC', 'bearing', 'pressure',
       'humidity', 'altitude', 'batteryPercent', 'eCO2', 'deviceId',
       'luminance', 'iaqAccuracy', 'temperature', 'userId']
initial_df = pd.DataFrame(columns = column_names)

for i in range(len(sensor_json['documents'])):
    df = pd.DataFrame(sensor_json['documents'][i]['fields'])
    df = df.ffill()
    df = df.backfill()
    df = df.drop(df.index[[1,2]])
    initial_df = initial_df.append(df)

In [121]:
#get the first 20 readings in the right df format 
initial_df

,serialNumber,iaq,timestamp,eVOC,bearing,pressure,humidity,altitude,batteryPercent,eCO2,deviceId,luminance,iaqAccuracy,temperature,userId
integerValue,m3wjc22wyh88,113.598,1598656211111,10.258130,104,96000,48.92491,453.0858,97,2248.63,fFrU8brLxN5Wea7PQwJg,67,3,24.81485,ztx11HwFoJPuDJd1p7lLilvvkjg2
doubleValue,VFC7uk23p7z,38.2759,1599949510904,0.674849,-140,97572,47.74441,317.2563,97,579.352,So15xXTTj0GAoVeomUCA,0,3,25.21985,ztx11HwFoJPuDJd1p7lLilvvkjg2
integerValue,m3wjc22wyh88,211.015,1598964608818,153.525300,108,96797,42.17585,383.9976,97,4038.6,fFrU8brLxN5Wea7PQwJg,2753,3,25.52985,ztx11HwFoJPuDJd1p7lLilvvkjg2
doubleValue,m3wjc22wyh88,64.4199,1599643221779,0.847608,110,97651,54.60197,310.4765,97,639.665,fFrU8brLxN5Wea7PQwJg,951,3,23.33985,ztx11HwFoJPuDJd1p7lLilvvkjg2
integerValue,NaN,25,1598266510410,0.500000,180,97025,38.03350,364.3173,0,500,SAGvj9Ar25aG4i3Xxta1,996,0,23.85000,ztx11HwFoJPuDJd1p7lLilvvkjg2
doubleValue,VFC7uk23p7z,50.8429,1600006214761,1.198076,-140,97682,39.16882,307.8174,97,828.089,So15xXTTj0GAoVeomUCA,1578,3,29.24485,ztx11HwFoJPuDJd1p7lLilvvkjg2
doubleValue,wthsftyK5vgu,125.617,1599690620121,3.240512,-28,97363,48.38392,335.2121,97,1486.32,UefkqbOojs07AR21i0ix,16,3,24.75485,ztx11HwFoJPuDJd1p7lLilvvkjg2
integerValue,fKuASomCxPUz,25,1600576518373,0.500000,152,96709,62.99857,391.6028,78,500,I1t2t86ZQoGIAv1i30r9,0,2,21.78985,ztx11HwFoJPuDJd1p7lLilvvkjg2
doubleValue,m3wjc22wyh88,113.798,1599976217116,1.222781,108,97784,61.69254,299.0734,97,841.591,fFrU8brLxN5Wea7PQwJg,334,3,20.88485,ztx11HwFoJPuDJd1p7lLilvvkjg2
doubleValue,wthsftyK5vgu,206.18,1601796315720,2.218808,-54,95306,56.89602,513.6255,65,1235.76,UefkqbOojs07AR21i0ix,332,1,19.13985,ztx11HwFoJPuDJd1p7lLilvvkjg2


In [103]:
#getting the json of page 2 
url = "https://firestore.googleapis.com/v1/projects/mimirhome-app/databases/(default)/documents/mimirReading"
page_token = sensor_json["nextPageToken"]
data2 = requests.get(url + f"?pageToken={page_token}").text
sensor_json2 = json.loads(data2)

In [105]:
column_names = ['serialNumber', 'iaq', 'timestamp', 'eVOC', 'bearing', 'pressure',
       'humidity', 'altitude', 'batteryPercent', 'eCO2', 'deviceId',
       'luminance', 'iaqAccuracy', 'temperature', 'userId']
initial_df2 = pd.DataFrame(columns = column_names)

for i in range(len(sensor_json2['documents'])):
    df = pd.DataFrame(sensor_json2['documents'][i]['fields'])
    df = df.ffill()
    df = df.backfill()
    df = df.drop(df.index[[1,2]])
    initial_df = initial_df.append(df)

In [108]:
initial_df

,serialNumber,iaq,timestamp,eVOC,bearing,pressure,humidity,altitude,batteryPercent,eCO2,deviceId,luminance,iaqAccuracy,temperature,userId
stringValue,VFC7uk23p7z,49.3138,1600401016692,0.922448,-136,97263,51.22694,343.8147,97,662.055,So15xXTTj0GAoVeomUCA,0,3,24.47485,ztx11HwFoJPuDJd1p7lLilvvkjg2
integerValue,dFp6ndvpmcg,25,1599697220536,0.500000,69,97259,55.45100,344.1588,97,500,gbeSfksDVEIUEEz60y87,0,0,23.22000,U6rvFKB7l8gOFQca1lMf8zP0i8X2
stringValue,wthsftyK5vgu,27.5749,1601711711278,0.522764,0,94931,60.04454,546.4860,75,511.781,UefkqbOojs07AR21i0ix,5,3,21.44485,ztx11HwFoJPuDJd1p7lLilvvkjg2
stringValue,fKuASomCxPUz,47.5516,1601937317123,0.702023,-174,96409,54.52648,417.5732,89,589.798,I1t2t86ZQoGIAv1i30r9,0,3,21.75485,ztx11HwFoJPuDJd1p7lLilvvkjg2
integerValue,m3wjc22wyh88,61.9662,1599177016369,1.234620,104,97779,49.36342,299.5024,97,847.965,fFrU8brLxN5Wea7PQwJg,0,3,23.76985,ztx11HwFoJPuDJd1p7lLilvvkjg2
doubleValue,fKuASomCxPUz,213.16,1602300022065,21.632970,-173,97190,62.03955,350.0990,52,2742.23,I1t2t86ZQoGIAv1i30r9,0,3,22.80485,ztx11HwFoJPuDJd1p7lLilvvkjg2
integerValue,fKuASomCxPUz,25,1601466887106,0.500000,176,96792,48.46700,384.4291,105,500,I1t2t86ZQoGIAv1i30r9,699,0,24.28000,ztx11HwFoJPuDJd1p7lLilvvkjg2
stringValue,VFC7uk23p7zk,58.6437,1600994714133,0.805453,-115,95541,54.19399,493.0860,74,626.166,So15xXTTj0GAoVeomUCA,0,3,20.97485,ztx11HwFoJPuDJd1p7lLilvvkjg2
doubleValue,m3wjc22wyh88,27.1344,1599815117419,0.523827,114,96951,50.22892,370.7006,97,512.319,fFrU8brLxN5Wea7PQwJg,2156,3,25.69485,ztx11HwFoJPuDJd1p7lLilvvkjg2
doubleValue,dFp6ndvpmcg,50.2399,1600648212829,0.590680,80,96672,57.19900,394.8026,97,544.102,gbeSfksDVEIUEEz60y87,0,1,24.19985,U6rvFKB7l8gOFQca1lMf8zP0i8X2


In [111]:
# getting all json files into df ---- this is the right thing!!!!!!!
url = "https://firestore.googleapis.com/v1/projects/mimirhome-app/databases/(default)/documents/mimirReading"
sensor_data = requests.get(url).text
column_names = ['serialNumber', 'iaq', 'timestamp', 'eVOC', 'bearing', 'pressure',
       'humidity', 'altitude', 'batteryPercent', 'eCO2', 'deviceId',
       'luminance', 'iaqAccuracy', 'temperature', 'userId']
initial_df = pd.DataFrame(columns = column_names)

while sensor_json["nextPageToken"]:
    page_token = sensor_json["nextPageToken"]
    data = requests.get(url + f"?pageToken={page_token}").text
    sensor_json = json.loads(data)
    for i in range(len(sensor_json['documents'])):
        df = pd.DataFrame(sensor_json['documents'][i]['fields'])
        df = df.ffill()
        df = df.backfill()
        df = df.drop(df.index[[1,2]])
        initial_df = initial_df.append(df)

KeyError: 'nextPageToken'

In [115]:
initial_df.to_csv('sensor_data.csv', index=False)

In [118]:
df = pd.read_csv('sensor_data.csv')

In [119]:
df 

,serialNumber,iaq,timestamp,eVOC,bearing,pressure,humidity,altitude,batteryPercent,eCO2,deviceId,luminance,iaqAccuracy,temperature,userId
0,fKuASomCxPUz,238.82520,1601676015176,5.668798,-159,94129,55.13599,617.1189,85,1856.2810,I1t2t86ZQoGIAv1i30r9,0,3,22.11985,ztx11HwFoJPuDJd1p7lLilvvkjg2
1,fKuASomCxPUz,25.00000,1601468131186,0.500000,171,96762,43.78288,387.0218,100,500.0000,I1t2t86ZQoGIAv1i30r9,699,0,26.01985,ztx11HwFoJPuDJd1p7lLilvvkjg2
2,fKuASomCxPUz,25.66664,1602404424738,0.507928,152,97294,54.54848,341.1472,48,504.1630,I1t2t86ZQoGIAv1i30r9,186,2,23.01985,ztx11HwFoJPuDJd1p7lLilvvkjg2
3,m3wjc22wyh88,54.74592,1599305715168,0.757911,103,97083,45.53238,359.3173,97,610.0677,fFrU8brLxN5Wea7PQwJg,1730,3,26.06485,ztx11HwFoJPuDJd1p7lLilvvkjg2
4,m3wjc22wyh88,105.09960,1600445101549,1.393267,112,97273,38.55331,342.9540,97,927.9365,fFrU8brLxN5Wea7PQwJg,2512,3,31.05985,ztx11HwFoJPuDJd1p7lLilvvkjg2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267,fKuASomCxPUz,25.00000,1600576518373,0.500000,152,96709,62.99857,391.6028,78,500.0000,I1t2t86ZQoGIAv1i30r9,0,2,21.78985,ztx11HwFoJPuDJd1p7lLilvvkjg2
1268,m3wjc22wyh88,113.79820,1599976217116,1.222781,108,97784,61.69254,299.0734,97,841.5912,fFrU8brLxN5Wea7PQwJg,334,3,20.88485,ztx11HwFoJPuDJd1p7lLilvvkjg2
1269,wthsftyK5vgu,206.18010,1601796315720,2.218808,-54,95306,56.89602,513.6255,65,1235.7600,UefkqbOojs07AR21i0ix,332,1,19.13985,ztx11HwFoJPuDJd1p7lLilvvkjg2
1270,m3wjc22wyh88,93.30641,1598736915134,0.784979,100,95970,58.39550,455.6953,97,619.3532,fFrU8brLxN5Wea7PQwJg,0,1,24.54485,ztx11HwFoJPuDJd1p7lLilvvkjg2
